### Importing the required packages and loading the data

In [ ]:
#importing the packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#Read in data into a dataframe
data = pd.read_csv(r"./Datasets/Energy_and_Water_Data.csv")

#Display top of dataframe
data.head()

In [ ]:
#No warnings about setting value on copy of slice
pd.options.mode.chained_assignment = None
#Diplay upto 60 columns of a dataset
pd.set_option('display.max_columns', 60)
#Set default font size
plt.rcParams['font.size'] = 10
sns.set(font_scale = 1)
#Internal ipython tool for setting figure size
from IPython.core.pylabtools import figsize

In [ ]:
#see the column data types and non-missing values
data.info()

### Convert data to correct types

In [ ]:
#Replace all occurances of Not Available with numpy not a number
data = data.replace({"Not Available":np.nan})

for col in list(data.columns):
    if ('ft²' in col or 'kBtu' in col or 'Metric Tons CO2e' in col or 'kWh' in 
        col or 'therms' in col or 'gal' in col or 'Score' in col):
        # Convert the data types to float
        data[col] = data[col].astype(float)

In [ ]:
#Statistics for each column
data.describe()

### Missing Values

In [ ]:
# Function to calculate missing values by column
def missing_values_table(df):
    #Total missing values
    mis_val = df.isnull().sum()
    
    #percentage of missing values
    mis_val_percent = 100 * df.isnull().sum()/len(df)
    
    mis_val_table = pd.concat([mis_val,mis_val_percent],axis=1)
    
    #Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(columns = {0:'Missing Values', 1 : '% of Total Values'})
    
    #Sort the table by percentage of missing descending
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] !=0
    ].sort_values('% of Total Values', ascending = False).round(1)
    
    #print some summary information
    print("Your selected dataframe has " + str(df.shape[1]) +
         " columns.\n"
         "There are " + str(mis_val_table_ren_columns.shape[0]) +
         " columns that have missing values.")
    # Return the dataframe with missing information
    return mis_val_table_ren_columns

In [ ]:
missing_values_table(data)

In [ ]:
missing_df = missing_values_table(data)
missing_columns = list(missing_df[missing_df['% of Total Values'] > 50].index)
print('we will remove %d columns.' % len(missing_columns))

In [ ]:
# Drop the columns
data = data.drop(columns = list(missing_columns))

### Imputing missing values

In [ ]:
numeric_subset = data.select_dtypes('number')
categorical_subset = data.select_dtypes('object')

In [ ]:
categorical_subset.columns

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
# Create an imputer object with a median filling strategy
num_imputer = SimpleImputer(strategy='median')

num_imputer.fit(numeric_subset)

num_data = num_imputer.transform(numeric_subset)
print("Missing values in numeric variables: ", np.isnan(num_data).sum())

In [ ]:
# Create an imputer object with a mode filling strategy
cat_imputer = SimpleImputer(strategy='most_frequent')

cat_imputer.fit(categorical_subset)

cat_data = cat_imputer.transform(categorical_subset)

In [ ]:
num_df = pd.DataFrame(num_data, columns =  numeric_subset.columns)

cat_df = pd.DataFrame(cat_data, columns = categorical_subset.columns)

mod_data = pd.concat([num_df, cat_df], axis=1)
mod_data.head()

In [ ]:
num_df = pd.DataFrame( num_data, columns =  numeric_subset.columns)
num_df.head()

In [ ]:
mod_data = pd.concat([num_df, cat_df], axis=1)
mod_data.head()

In [ ]:
mod_data.isnull().sum().sum()

### Exploratory Data Analysis

In [ ]:
plt.figure(figsize=(8,8))

# Rename the socre
data = data.rename(columns = {'ENERGY STAR Score' : 'Score'})

# Histogram of the Energy Star Score
plt.style.use('fivethirtyeight')
plt.hist(data['Score'].dropna(), bins = 100, edgecolor = 'k')
plt.xlabel('Score')
plt.ylabel('Number of Buildings')
plt.title('Energy Star Score Distribution')

In [ ]:
# Histogram plot of site EUI
plt.figure(figsize=(8,8))
plt.hist(data['Site EUI (kBtu/ft²)'].dropna(),bins=20,edgecolor='black')
plt.xlabel('Site EUI')
plt.ylabel('Count')
plt.title('Site EUI Distribution')

In [ ]:
data['Site EUI (kBtu/ft²)'].describe()

In [ ]:
data['Site EUI (kBtu/ft²)'].dropna().sort_values(ascending = False).head(10)

In [ ]:
data.loc[data['Site EUI (kBtu/ft²)'] == 869265.0, :]

In [ ]:
# Calculate first and third quartile
first_quantile = data['Site EUI (kBtu/ft²)'].describe()['25%']
third_quantile = data['Site EUI (kBtu/ft²)'].describe()['75%']

# Interquartile range
iqr = third_quantile - first_quantile

#Remove outliers
data = data[(data['Site EUI (kBtu/ft²)'] > (first_quantile - 3 * iqr)) & 
            (data['Site EUI (kBtu/ft²)'] < (third_quantile + 3 * iqr))]

In [ ]:
# Histogram plot of site EUI
plt.figure(figsize=(8,8))
plt.hist(data['Site EUI (kBtu/ft²)'].dropna(),bins=20,edgecolor='black')
plt.xlabel('Site EUI')
plt.ylabel('Count')
plt.title('Site EUI Distribution')

In [ ]:
data['Site EUI (kBtu/ft²)'].describe()

In [ ]:
for col in cat_df:
    print('\nFrequency of categories for variable %s'%col)
    print(cat_df[col].value_counts())

In [ ]:
types = data.dropna(subset=['Score'])
types = types['Largest Property Use Type'].value_counts()
types = list(types[types.values > 100].index)
types

In [ ]:
# Plot of distribution of scores for building categories

plt.figure(figsize=(12,10))

# plot each building
for b_type in types:
    #select the building type
    subset = data[data['Largest Property Use Type'] == b_type]
    # Density plot of Energy Star Scores
    sns.kdeplot(subset['Score'].dropna(),
               label = b_type, shade = False,
               alpha = 0.8)

# label the plot
plt.xlabel('Energy Star Score', size = 25)
plt.ylabel('Density', size = 25); 
plt.title('Density Plot of Energy Star Scores by Building Type', size = 25);

In [ ]:
# Create a list of boroughs with more than 100 observations
boroughs = data.dropna(subset=['Score'])
boroughs = boroughs['Borough'].value_counts()
boroughs = list(boroughs[boroughs.values > 100].index)
boroughs

In [ ]:
# Plot of distribution scores of boroughs

plt.figure(figsize=(8,8))

# Plot each borough
for b_borough in boroughs:
    subset = data[data['Borough'] == b_borough]
    sns.kdeplot(subset['Score'].dropna(),
               label = b_borough, shade = False,
               alpha = 0.8)

plt.xlabel("Energy Star Score")
plt.ylabel("Density")
plt.title("Density plot of Energy Star Score by Borough")

In [ ]:
# Find all correlations and sort
correlations_data = data.corr()['Score'].sort_values()

# Print the most negative correlations
print(correlations_data.head(15), '\n')

# Print the most positive correlations
print(correlations_data.tail(15))

In [ ]:
numeric_subset = data.select_dtypes('number')

# Create columns with square root and log of numeric columns
for col in numeric_subset.columns:
    # Skip the Energy Star Score column
    if col == 'Score':
        next
    else:
        numeric_subset['sqrt_' + col] = np.sqrt(numeric_subset[col])
        numeric_subset['log_' + col] = np.log(numeric_subset[col])

# Select the categorical columns
categorical_subset = data[['Borough', 'Largest Property Use Type']]

# One hot encode
categorical_subset = pd.get_dummies(categorical_subset)

# Join the two dataframes using concat
# Make sure to use axis = 1 to perform a column bind
features = pd.concat([numeric_subset, categorical_subset], axis = 1)

# Drop buildings without an energy star score
features = features.dropna(subset = ['Score'])

# Find correlations with the score 
correlations = features.corr()['Score'].dropna().sort_values()

In [ ]:
# Display most negative correlations
correlations.head(15)

In [ ]:
# Display most positive correlations
correlations.tail(15)

In [ ]:
plt.figure(figsize=(8,8))

# Extract the building types
features['Largest Property Use Type'] = data.dropna(subset = ['Score'])['Largest Property Use Type']

# Limit to building types with more than 100 observations
features = features[features['Largest Property Use Type'].isin(types)]

# Use seaborn to plot a scatterplot of Score Vs Log Source EUI
sns.lmplot('Site EUI (kBtu/ft²)', 'Score', 
           hue = 'Largest Property Use Type',
          data = features,
          scatter_kws = {'alpha':0.8, 's':60},
          fit_reg = False, size = 12, aspect = 1.2)

# Plot labeling
plt.xlabel("Site EUI", size = 28)
plt.ylabel("Energy Star Score", size = 28)
plt.title("Energy Star Score vs Site EUI", size = 36)

In [ ]:
# Extract the columns to  plot
plot_data = features[['Score', 'Site EUI (kBtu/ft²)', 
                      'Weather Normalized Source EUI (kBtu/ft²)', 
                      'log_Total GHG Emissions (Metric Tons CO2e)']]

# Replace the inf with nan
plot_data = plot_data.replace({np.inf: np.nan, -np.inf: np.nan})

# Rename columns 
plot_data = plot_data.rename(columns = {'Site EUI (kBtu/ft²)': 'Site EUI', 
                                        'Weather Normalized Source EUI (kBtu/ft²)': 'Weather Norm EUI',
                                        'log_Total GHG Emissions (Metric Tons CO2e)': 'log GHG Emissions'})

# Drop na values
plot_data = plot_data.dropna()

# Function to calculate correlation coefficient between two columns
def corr_func(x, y, **kwargs):
    r = np.corrcoef(x, y)[0][1]
    ax = plt.gca()
    ax.annotate("r = {:.2f}".format(r),
               xy=(.2, .8), xycoords = ax.transAxes,
               size = 10)

# Create the pairgrid object
grid = sns.PairGrid(data = plot_data, size = 3)

# Upper is a scatter plot
grid.map_upper(plt.scatter, color = 'blue', alpha = 0.6)

# Diagonal is a histogram
grid.map_diag(plt.hist, color = 'blue', edgecolor = 'black')

# Bottom is correlation and density plot
grid.map_lower(corr_func)
grid.map_lower(sns.kdeplot,cmap = plt.cm.Reds)

#Title for entire plot
plt.suptitle('Pairs Plot of Engery Data', size = 25, y = 1.02)

In [ ]:
corr = plot_data.corr()
f, ax = plt.subplots(figsize=(8, 8))
sns.heatmap(corr, vmax=1,annot_kws={'size': 15}, annot=True);

In [ ]:
data

In [ ]:
# we will limit the graph to building types that have 
# more than 100 observations in the dataset.
building_types = data.dropna(subset=['Score'])
building_types = building_types['Largest Property Use Type'].value_counts()
building_types = list(building_types[building_types.values > 100].index)
print("Buidling types with more than 100 observations ",building_types)

# Create a list of boroughs with more than 100 observations
boroughs = data.dropna(subset=['Score'])
boroughs = boroughs['Borough'].value_counts()
boroughs = list(boroughs[boroughs.values > 100].index)
print("Boroughs with more than 100 observations ",boroughs)

In [ ]:
multivari_data = data[data['Largest Property Use Type'].isin(building_types) & 
                      data['Borough'].isin(boroughs)].dropna()
multivari_data.rename(columns = {'Largest Property Use Type':"BuildingType"}, 
                      inplace = True)
multivari_data.head()

In [ ]:
plt.figure(figsize=(20,12))
x=sns.FacetGrid(multivari_data, row='Borough',col = 'BuildingType',
                palette='husl',sharex=False,sharey=False, margin_titles=True)
x=x.map(plt.hist, 'Score', bins=15)
x=x.fig.subplots_adjust(wspace=0.5, hspace=0.5)

### Program assignment solution

In [ ]:
#importing the data
MTcars = pd.read_csv("./Datasets/mtcars.csv", sep=",")

In [ ]:
MTcars.head()

In [ ]:
MTcars['cyl'] = MTcars['cyl'].astype('object')
MTcars['vs'] = MTcars['am'].astype('object')
MTcars['am'] = MTcars['am'].astype('object')
MTcars['gear'] = MTcars['gear'].astype('object')
MTcars['carb'] = MTcars['carb'].astype('object')

In [ ]:
MTcars.shape

In [ ]:
# Distribution of mpg variable
plt.hist(MTcars['mpg'], bins = 7, edgecolor = 'k')
plt.xlabel('Mpg')
plt.ylabel('Number of kilometers')
plt.title('Miles/US Gallon')

In [ ]:
MTcars.corr()

In [ ]:
cor = MTcars.corr()
f, ax = plt.subplots(figsize=(8, 8))
sns.heatmap(cor, vmax=1,annot_kws={'size': 15}, annot=True);

In [ ]:
MTcars.info()

In [ ]:
plt.figure(figsize=(20,12))
x=sns.FacetGrid(MTcars, col='cyl',
                palette='husl',sharex=False,sharey=False, margin_titles=True)
x=x.map(plt.hist, 'mpg', bins=15)
x=x.fig.subplots_adjust(wspace=0.5, hspace=0.5)

In [ ]:
plt.figure(figsize=(20,12))
x=sns.FacetGrid(MTcars, col='am',
                palette='husl',sharex=False,sharey=False, margin_titles=True)
x=x.map(plt.hist, 'mpg', bins=15)
x=x.fig.subplots_adjust(wspace=0.5, hspace=0.5)

In [ ]:
plt.figure(figsize=(20,12))
x=sns.FacetGrid(MTcars, col='gear',
                palette='husl',sharex=False,sharey=False, margin_titles=True)
x=x.map(plt.hist, 'mpg', bins=15)
x=x.fig.subplots_adjust(wspace=0.5, hspace=0.5)